In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

from eval import evaluator, Task
from eval.eval_configs import NumericConfig

/Users/dhruvkapur/anaconda3/envs/taskweaver/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.mps.is_available() else 'cpu'
print(device)

mps


In [3]:
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen3-0.6B', device_map='auto')
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-0.6B', padding_side='left')

In [4]:
dataset = load_dataset('openai/gsm8k', 'main', split='test[:100]')
eval_config = NumericConfig(tolerance=0)
user_template = "{question}"
assistant_template = "{answer}"
system_prompt = "Solve the given math problem by thinking step by step."

gsm8k_task = Task('gsm8k', dataset, user_template, assistant_template, system_prompt, eval_config)

Map: 100%|██████████| 100/100 [00:00<00:00, 28439.82 examples/s]


In [5]:
result = gsm8k_task.evaluate(model, tokenizer, batch_size=8, progress=True)

100%|██████████| 13/13 [09:03<00:00, 41.84s/it]


In [6]:
result

EvaluationResult(NUMERICAL, accuracy=0.0800, answered_rate=1.0000, correct=8, answered=100, total=100, n=100)

In [7]:
print(result.predictions[0])

To find out how much Janet makes every day at the farmers' market, we need to break down the problem into parts and calculate each component step by step.

---

### **Step 1: Total eggs laid per day**
Janet’s ducks lay **16 eggs per day**.

---

### **Step 2: Eggs eaten for breakfast**
She eats **3 eggs** for breakfast every morning.

So, **3 eggs** are eaten each day.

---

### **Step 3: Eggs used for baking muffins**
She bakes **4 eggs** for her friends every day.

So, **4 eggs** are used for baking muffins.

---

### **Step 4: Eggs left for the market**
The remaining eggs are:
$$
16 \text{ (laid per day)} - 3 \text{ (eaten for breakfast)} - 4 \text{ (baked for muffins)} = 9 \text{ eggs}
$$

---

### **Step 5: Selling eggs at the market**
She sells the remaining **9 eggs** at **$2 per egg**.

So, the total money made at the market is:
$$
9 \times 2 = 18 \text{ dollars}
$$

---

### ✅ **Final Answer:**
$$
\boxed{18} \text{ dollars}
$$


In [9]:
print(result.references[0])

Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


In [10]:
print(dataset[0]['question'])

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
